In [1]:
from pathlib import Path
import pandas as pd
import xlwings as xw
import numpy as np
import matplotlib.pyplot as plt
from rdp import rdp

In [2]:
current_dir = Path.cwd()
parent_dir = current_dir.parent
input_dir1 = Path ("G:/Chamodi/Machine_Learning/Pressure_history_2/Pressure_profiles_arrival_omitted")
#input_dir1 = Path ("C:/Users/z5379606/OneDrive - UNSW/Documents/LSDYNA_files/Incident_pressure/Incident_pressure_time_history/TNT_far_field")

#Access folders inside input folder
input_dir2 =  [folder for folder in input_dir1.iterdir() if folder.is_dir()]
print ("2",input_dir2)

# Create output directory
output_dir1 = Path("G:/Chamodi/LSDYNA_1D/Incident_pressure_time_history/TNT_far_field")
output_dir1.mkdir(exist_ok = True)

# Access the template to be used to generate files
excel_template = current_dir/ "Template.xlsx"

2 [WindowsPath('G:/Chamodi/Machine_Learning/Pressure_history_2/Pressure_profiles_arrival_omitted/L13_1'), WindowsPath('G:/Chamodi/Machine_Learning/Pressure_history_2/Pressure_profiles_arrival_omitted/L13_11'), WindowsPath('G:/Chamodi/Machine_Learning/Pressure_history_2/Pressure_profiles_arrival_omitted/L13_13'), WindowsPath('G:/Chamodi/Machine_Learning/Pressure_history_2/Pressure_profiles_arrival_omitted/L13_15'), WindowsPath('G:/Chamodi/Machine_Learning/Pressure_history_2/Pressure_profiles_arrival_omitted/L13_17'), WindowsPath('G:/Chamodi/Machine_Learning/Pressure_history_2/Pressure_profiles_arrival_omitted/L13_19'), WindowsPath('G:/Chamodi/Machine_Learning/Pressure_history_2/Pressure_profiles_arrival_omitted/L13_21'), WindowsPath('G:/Chamodi/Machine_Learning/Pressure_history_2/Pressure_profiles_arrival_omitted/L13_23'), WindowsPath('G:/Chamodi/Machine_Learning/Pressure_history_2/Pressure_profiles_arrival_omitted/L13_25'), WindowsPath('G:/Chamodi/Machine_Learning/Pressure_history_2/Pr

In [3]:
def reduce_points(time, pressure):
    
    zero_crossing_index = next((i for i, val in enumerate(pressure[:], start=0) if val <= 0), None)
    
    # Combine time and pressure into a single array of 2D points
    points = np.column_stack((time, pressure))
    
    segment1 = points[:zero_crossing_index]  # Example segmentation
    segment2 = points[zero_crossing_index:]
    
    reduced_segment1 = rdp(segment1, epsilon=0.001)
    reduced_segment2 = rdp(segment2, epsilon=0.01)
    
    reduced_points = np.vstack((reduced_segment1, reduced_segment2))
    
    # Separate the reduced points into time and pressure
    reduced_time = reduced_points[:, 0]
    reduced_pressure = reduced_points[:, 1]
    
    return reduced_time, reduced_pressure

In [4]:
def get_phase_array(time, pressure, CT, CP):
    a=0
    b=0
    phase = []
    for t, p in zip(time, pressure):
        if round (t, 10) < round (CT, 10):
            phase.append('positive')#{t} {change_time}')
        elif round (CT, 10) <= round (t, 10):
            a+=1
            if a == 1 and abs(p - CP) <0.000001:
                phase.append('negative')
            elif a == 1 and abs(p - CP) >0.000001:
                phase.append('error')
                print (f'error- p={p} , change p ={CP}, t ={t}, change t = {CT}')
            else:
                phase.append('negative')
        else:
            phase.append('something wrong')
            print ('something wrong')
                
    return phase

In [5]:
def find_CE(reduced_pressure, reduced_time, file1, EM, SD):
    
    # Find the first zero crossing between peak and lowest value
    zero_crossing_index_1 = next((i for i, val in enumerate(reduced_pressure[:], start=0) if val <= 0), None)
    
    # CHANGE AND END
    CT = reduced_time[zero_crossing_index_1]
    CP = reduced_pressure[zero_crossing_index_1]
    ET = reduced_time[-1]
    EP = reduced_pressure[-1]
    
    #print ('CT', CT, type(CT), 'CP', CP, type(CP), 'TT', TT, type(TT), 'TP', TP, type(TP))
    print (zero_crossing_index_1)
    
    if all(isinstance(val, np.floating) for val in [CT, CP]):
    
        phase = get_phase_array(reduced_time, reduced_pressure, CT, CP)
   
        print('phase len', len(phase))
    
        # Create a dictionary with the arrays
        data = {"Mass": [EM] * len(reduced_time),
                 "Standoff distance": [SD] * len(reduced_time),
                 'Time': reduced_time,
                 "Phase": phase,
                 'Pressure': reduced_pressure}
        
        # Create DataFrames from dictionaries
        df2 = pd.DataFrame(data)
        
        # Initiate xlwings library
        with xw.App (visible = False) as app:
            
            wb = app.books.open(excel_template)
            # Write dataframe in excel template
            wb.sheets[0].range("A1").options(index=False).value = df2
            
            # Create files in output directory
            wb.save(output_dir2/f"{file1.stem}.xlsx")
            wb.close()
        
        print ("file_name",file1.stem)
    else:
        CT = CT if isinstance(CT, (np.floating, float)) else None
        CP = CP if isinstance(CP, (np.floating, float)) else None
    
    return CT, CP, ET, EP

In [6]:
df_all = pd.DataFrame(columns=["Mass", "Standoff distance", "Arrival time", "AP", "Change time", "CP", "End time", "EP","Original", "Reduced" ])

for folder1 in input_dir2: # folder1-L13_1

    # Create output folders
    output_dir2 = output_dir1/ folder1.name
    output_dir2.mkdir(exist_ok = True)
    print ("OUTPUT 2", output_dir2)
    
    # EM
    parts = folder1.name.split('_')
    
    EM = round(int(parts[1])/2, 1)
        
    # Make a list of data file names
    excel_files = list(folder1.rglob("*.xlsx"))
    
    # Access each file 
    for file1 in excel_files: # file-1.xlsx
        
        # Assign value to A based on file.stem
        file_name = file1.stem
        SD = round((int(file_name[1:])+1)/2,0)
        
        # Read data from Excel file
        df1 = pd.read_excel(file1)
        
        # Extract time and pressure columns
        time = df1["Time"].dropna().values*1000
        pressure = (df1["Pressure"].dropna().values/1000)-101

        # ARRIVAL
        AT = time[0]
        AP = pressure[0]
                
        reduced_time, reduced_pressure = reduce_points(time, pressure)

        CT, CP, ET, EP = find_CE(reduced_pressure, reduced_time, file1, EM, SD)
            

        # Creating a new DataFrame to append
        row_all = pd.DataFrame({"Mass": [EM], "Standoff distance": [SD],
                                "Arrival time": [AT],"AP": [AP],
                                "Change time": [CT], "CP": [CP],
                                "End time": [ET], "EP": [EP],
                                "Original":[len(pressure)], "Reduced": [len(reduced_pressure)]})
        df_all = pd.concat([df_all, row_all], ignore_index=True)

        # Create the plot
        plt.figure(figsize=(4, 4))
        plt.plot(time, pressure, color='b', linestyle=':')
        plt.plot(reduced_time, reduced_pressure, color='r', linestyle='-', marker='o')
        # Save the plot as an image
        plt.savefig(output_dir2/f'{file1.stem}.png', dpi=150, bbox_inches='tight', facecolor='white')
            
        # Close the plot to free up memory
        plt.close()                 

OUTPUT 2 G:\Chamodi\LSDYNA_1D\Incident_pressure_time_history\TNT_far_field\L13_1
26
phase len 115
file_name P1
32
phase len 84
file_name P13
35
phase len 82
file_name P17
35
phase len 76
file_name P21
45
phase len 85
file_name P25
48
phase len 84
file_name P29
49
phase len 82
file_name P33
54
phase len 86
file_name P37
57
phase len 86
file_name P41
65
phase len 94
file_name P45
70
phase len 99
file_name P49
32
phase len 107
file_name P5
71
phase len 99
file_name P53
66
phase len 92
file_name P57
85
phase len 112
file_name P59
29
phase len 92
file_name P9
OUTPUT 2 G:\Chamodi\LSDYNA_1D\Incident_pressure_time_history\TNT_far_field\L13_11
10
phase len 182
file_name P1
34
phase len 128
file_name P13
34
phase len 115
file_name P17
36
phase len 109
file_name P21
36
phase len 102
file_name P25
39
phase len 101
file_name P29
41
phase len 98
file_name P33
43
phase len 96
file_name P37
52
phase len 101
file_name P41
50
phase len 99
file_name P45
50
phase len 98
file_name P49
36
phase len 152
file

In [7]:
# Write dataframes in excel template
all_path = 'IP_far_field_all_reduced.xlsx'

df_all.to_excel(all_path, index=False)